In [16]:
import pandas as pd

# Data reorganization

In [17]:
data = pd.read_json('data/merged.jsonl', lines=True)
data.columns

Index(['session_id', 'timestamp', 'user_id', 'product_id', 'event_type',
       'offered_discount', 'purchase_id', 'made_purchase', 'known_product',
       'product_name', 'price', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'city',
       'street', 'name_1', 'name_2', 'surname', 'female', 'purchase_timestamp',
       'delivery_timestamp', 'delivery_company', 'delivery_time'],
      dtype='object')

In [19]:
data.columns = ['session_id', 'time', 'user_id', 'product_id', 'event',\
                'discount', 'purchase_id', 'made_purchase', 'known_product',\
                'product', 'price', 'cat_0', 'cat_1', 'cat_2', 'cat_3',\
                'city', 'street', 'name_1', 'name_2', 'surname', 'female',\
                'purchase_time', 'delivery_time', 'delivery_company', 'delivery_time']
data.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,product,...,city,street,name_1,name_2,surname,female,purchase_time,delivery_time,delivery_company,delivery_time
0,100001,2020-01-17 16:08:57,102.0,1001,VIEW_PRODUCT,0,NaN,False,True,Telefon Siemens Gigaset DA310,...,Warszawa,plac Krakowska 69,Juliusz,None,Męcik,0.0,NaN,NaN,NaN,NaT
1,100002,2020-01-07 05:10:41,102.0,1277,VIEW_PRODUCT,20,NaN,False,True,Apple iPad mini 64GB 4G,...,Warszawa,plac Krakowska 69,Juliusz,None,Męcik,0.0,NaN,NaN,NaN,NaT
2,100002,2020-01-07 05:14:26,102.0,1276,VIEW_PRODUCT,20,NaN,False,True,Apple iPad mini 64GB,...,Warszawa,plac Krakowska 69,Juliusz,None,Męcik,0.0,NaN,NaN,NaN,NaT
3,100003,2020-01-16 09:37:42,102.0,1276,VIEW_PRODUCT,0,NaN,False,True,Apple iPad mini 64GB,...,Warszawa,plac Krakowska 69,Juliusz,None,Męcik,0.0,NaN,NaN,NaN,NaT
4,100003,2020-01-16 09:38:22,102.0,1277,VIEW_PRODUCT,0,NaN,False,True,Apple iPad mini 64GB 4G,...,Warszawa,plac Krakowska 69,Juliusz,None,Męcik,0.0,NaN,NaN,NaN,NaT


# Distribution analysis